# UpstashVectorStore

[Upstash Vector](https://upstash.com/) 是一个基于 REST 的无服务器向量数据库，专为处理向量嵌入而设计。

本指南提供了关于如何快速开始使用 Upstash [向量存储](/docs/concepts/#vectorstores) 的简要概述。如需了解 `UpstashVectorStore` 所有功能和配置的详细文档，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain_community_vectorstores_upstash.UpstashVectorStore.html)。

## 概览

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/upstash/) | 包的最新版本 |
| :--- | :--- | :---: | :---: |
| [`UpstashVectorStore`](https://api.js.langchain.com/classes/langchain_community_vectorstores_upstash.UpstashVectorStore.html) | [`@langchain/community`](https://npmjs.com/@langchain/community) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 环境配置

要使用 Upstash 向量存储，你需要创建一个 Upstash 账户，创建一个索引，并安装 `@langchain/community` 集成包。你还需要将 [`@upstash/vector`](https://www.npmjs.com/package/@upstash/vector) 包作为对等依赖安装。

本指南还将使用 [OpenAI 嵌入](/docs/integrations/text_embedding/openai)，这需要你安装 `@langchain/openai` 集成包。如果你愿意，也可以使用[其他受支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core @upstash/vector @langchain/openai
</Npm2Yarn>
```

你可以通过 [Upstash 控制台](https://console.upstash.com/login) 创建一个索引。更多参考信息，请参阅 [官方文档](https://upstash.com/docs/vector/overall/getstarted)。

Upstash 向量还内置了嵌入支持。这意味着你可以直接使用它，而无需额外的嵌入模型。更多详细信息，请查看 [嵌入模型文档](https://upstash.com/docs/vector/features/embeddingmodels)。

```{=mdx}
:::note
要使用内置的 Upstash 嵌入，你需要在创建索引时选择一个嵌入模型。
:::
```

### 凭证

设置好索引后，请设置以下环境变量：

```typescript
process.env.UPSTASH_VECTOR_REST_URL = "your-rest-url";
process.env.UPSTASH_VECTOR_REST_TOKEN = "your-rest-token";
```

如果你在本指南中使用 OpenAI 嵌入，还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想对模型调用进行自动化追踪，也可以通过取消以下注释来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

确保你的索引具有与你的嵌入向量相同的维度数量。OpenAI `text-embedding-3-small` 的默认维度是 1536。

In [1]:
import { UpstashVectorStore } from "@langchain/community/vectorstores/upstash";
import { OpenAIEmbeddings } from "@langchain/openai";

import { Index } from "@upstash/vector";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const indexWithCredentials = new Index({
  url: process.env.UPSTASH_VECTOR_REST_URL,
  token: process.env.UPSTASH_VECTOR_REST_TOKEN,
});

const vectorStore = new UpstashVectorStore(embeddings, {
  index: indexWithCredentials,
  // You can use namespaces to partition your data in an index
  // namespace: "test-namespace",
});

## 使用内置嵌入

要使用内置的 Upstash 嵌入，可以将 `FakeEmbeddings` 实例传递给 `UpstashVectorStore` 构造函数。这将使 `UpstashVectorStore` 使用内置的嵌入，该嵌入在创建索引时已由你选择。

In [ ]:
import { UpstashVectorStore } from "@langchain/community/vectorstores/upstash";
import { FakeEmbeddings } from "@langchain/core/utils/testing";

import { Index } from "@upstash/vector";

const indexWithEmbeddings = new Index({
  url: process.env.UPSTASH_VECTOR_REST_URL,
  token: process.env.UPSTASH_VECTOR_REST_TOKEN,
});

const vectorStore = new UpstashVectorStore(new FakeEmbeddings(), {
  index: indexWithEmbeddings,
});

## 管理向量存储

### 向向量存储中添加条目

In [2]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents, { ids: ["1", "2", "3", "4"] });

[ '1', '2', '3', '4' ]


**注意：** 添加文档后，可能会有短暂延迟才能进行查询。

### 从向量存储中删除项目

In [3]:
await vectorStore.delete({ ids: ["4"] });

## 查询向量存储

一旦创建了向量存储并添加了相关文档，在运行链或代理时很可能需要对其进行查询。

### 直接查询

执行一个简单的相似性搜索可以按照以下方式：

In [7]:
const filter = "source = 'https://example.com'";

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


更多关于Upstash Vector过滤语法的内容请参见[此页面](https://upstash.com/docs/vector/features/filtering)。

如果你想执行相似性搜索并获得相应的分数，可以运行：

In [8]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.576] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.557] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### 通过转换为检索器进行查询

你还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在你的链中更方便地使用。

In [9]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [如何：使用RAG进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## API 参考

有关所有 `UpstashVectorStore` 功能和配置的详细文档，请访问 [API 参考](https://api.js.langchain.com/classes/langchain_community_vectorstores_upstash.UpstashVectorStore.html)。